In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
import torch
import os
import os.path as osp
import seaborn as sb
from matplotlib import pyplot as plt
import glob
from yaml import Loader, FullLoader, load
import pickle5 as pickle
from matplotlib import cm
import itertools as it
import networkx as nx
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
from pyproj import Proj, transform
from shapely import geometry
import geoplot as gplt
from matplotlib.ticker import FixedLocator

/home/flipper/anaconda3/envs/birdmigration/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [170]:
def plot_performance(ax, df, label, metric, color, static=False, H=64, ls='standard', z=1, night=False):
    
    xscale = 'night_horizon' if night else 'horizon'
    
    df = df.query(f'{xscale} > 0 & {xscale} <= {H}')
    
    if static:
        df = df.groupby('trial').aggregate(np.nanmean).reset_index()
        avg = np.array([np.nanmean(df[metric].values)]*H)
        std = np.array([np.nanstd(df[metric].values)]*H)
        ls = '--' if ls == 'standard' else ls
    else:
        grouped = df.groupby(xscale)        
        avg = grouped[metric].aggregate(np.nanmean).values #[-H:]
        std = grouped[metric].aggregate(np.nanstd).values #[-H:]
        ls = '-' if ls == 'standard' else ls
    
    h_range = df[xscale].unique()
    
    line = ax.plot(h_range, avg, label=label, ls=ls, color=color, linewidth=1.8, zorder=z)
    ax.fill_between(h_range, avg-std, avg+std, color=color, alpha=0.2, zorder=z)

In [112]:
home = osp.expanduser("~")
base_dir = osp.join(home, 'FluxRGNN')
result_dir = osp.join(base_dir, 'results')
output_dir = osp.join(base_dir, 'data', 'plots', 'final')
os.makedirs(output_dir, exist_ok=True)

In [113]:
C, H = 24, 72
test_year = 2017

# load configs for abm and radar experiments
bscale = {}
abm_cfg = osp.join(result_dir, 'abm', 'GBT', f'test_{test_year}', 
                   'final', 'trial_1', 'config.yaml')
with open(abm_cfg) as f:
    config = load(f, Loader=FullLoader)
    bscale['abm'] = config['datasource']['bird_scale']
    
radar_cfg = osp.join(result_dir, 'radar', 'GBT', f'test_{test_year}', 
                     'final', 'trial_1', 'config.yaml')
with open(radar_cfg) as f:
    config = load(f, Loader=FullLoader)
    bscale['radar'] = config['datasource']['bird_scale']

In [175]:
# define baseline models and their labels
baseline_models = {('HA', 'final'): 'HA',
          ('GAM', 'final'): 'GAM',
          ('GBT', 'final'): 'GBT'}

baseline_models = {('HA', 'final'): 'Historical average',
          ('GAM', 'final'): 'Generalized additive model',
          ('GBT', 'final'): 'Gradient boosted regression trees'}
# define FluxRGNN-type models and their labels
flux_models = {    
          ('FluxRGNN', 'final'): 'FluxRGNN', 
          ('LocalLSTM', 'final'): 'w/o fluxes',       
          ('FluxRGNN', 'final_without_encoder'): 'w/o encoder',
          ('FluxRGNN', 'final_without_boundary'): 'w/o boundary'}
# define colors
baseline_colors = ['#cccccc', '#999999', '#404040']
baseline_ls = ['--', '-.', ':']
flux_colors = ['#ff7f00', '#b30000', '#999966', '#008080']

### ***Predictive performance***

In [154]:
fixed_t0 = False
ext = '_fixedT0' if fixed_t0 else ''

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(14,7), sharex=True)

for i, datasource in enumerate(['abm', 'radar']):
    
    # load baseline performance
    baseline_rmse = pd.read_csv(osp.join(result_dir, datasource, 
                    f'performance_evaluation{ext}', 'final', 'rmse_per_hour.csv'))
    baseline_pcc = pd.read_csv(osp.join(result_dir, datasource, 
                    f'performance_evaluation{ext}', 'final', 'pcc_per_hour.csv'))
    baseline_rmse['rmse'] /= bscale[datasource]
    
    # load FluxRGNN performance
    rmse_per_hour = pd.read_csv(osp.join(result_dir, datasource, 
                    f'performance_evaluation{ext}', 'ablations', 'rmse_per_hour.csv'))
    pcc_per_hour = pd.read_csv(osp.join(result_dir, datasource, 
                    f'performance_evaluation{ext}', 'ablations', 'pcc_per_hour.csv'))
    rmse_per_hour['rmse'] /= bscale[datasource]


    for j, ((m, e), label) in enumerate(baseline_models.items()):
        plot_performance(ax[0, i], baseline_rmse.query(f'model == "{m}" & experiment == "{e}"'), 
                         label, 'rmse', H=H, color=baseline_colors[j], ls=baseline_ls[j])
        plot_performance(ax[1, i], baseline_pcc.query(f'model == "{m}" & experiment == "{e}"'), 
                         label, 'pcc', H=H, color=baseline_colors[j], ls=baseline_ls[j])

    for j, ((m, e), label) in enumerate(flux_models.items()):
        if m == 'FluxRGNN' and e == 'final':
            z = 2
        else:
            z = 1
        plot_performance(ax[0, i], rmse_per_hour.query(f'model == "{m}" & experiment == "{e}"'), 
                         label, 'rmse', H=H, color=flux_colors[j], z=z)
        plot_performance(ax[1, i], pcc_per_hour.query(f'model == "{m}" & experiment == "{e}"'), 
                         label, 'pcc', H=H, color=flux_colors[j], z=z)

    ax[1,i].set_xlabel('forcasting horizon [h]', fontsize=14)
    
ax[0,0].set_ylabel('RMSE', fontsize=14)
ax[1,0].set_ylabel('Pearson r', fontsize=14)
ax[1,0].legend(loc='upper right', fontsize=12, 
            bbox_to_anchor=(0.97,1.37), framealpha=1)
ax[0,0].set_title('Simulated data', fontsize=14)
ax[0,1].set_title('Radar data', fontsize=14)

for axis in ax.flatten():
    axis.tick_params(axis='both', which='major', labelsize=12)
    axis.grid(color = 'gray', linestyle = '-', alpha=0.2)

fig.subplots_adjust(wspace=0.25, hspace=0)
fig.align_ylabels(ax)
fig.savefig(osp.join(output_dir, f'final_rmse_pcc_per_hour{ext}.png'), bbox_inches='tight', dpi=200)

In [173]:
H=64

fixed_t0 = True
ext = '_fixedT0' if fixed_t0 else ''

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16,7), sharex=True)

for i, datasource in enumerate(['abm', 'radar']):
    
    # load baseline performance
    baseline_rmse = pd.read_csv(osp.join(result_dir, datasource, 
                    f'performance_evaluation{ext}', 'final', 'rmse_per_hour.csv'))
    baseline_pcc = pd.read_csv(osp.join(result_dir, datasource, 
                    f'performance_evaluation{ext}', 'final', 'pcc_per_hour.csv'))
    baseline_rmse['rmse'] /= bscale[datasource]
    
    # load FluxRGNN performance
    rmse_per_hour = pd.read_csv(osp.join(result_dir, datasource, 
                    f'performance_evaluation{ext}', 'ablations', 'rmse_per_hour.csv'))
    pcc_per_hour = pd.read_csv(osp.join(result_dir, datasource, 
                    f'performance_evaluation{ext}', 'ablations', 'pcc_per_hour.csv'))
    rmse_per_hour['rmse'] /= bscale[datasource]

    for j, ((m, e), label) in enumerate(flux_models.items()):
        if e == 'final':
            plot_performance(ax[0, i], rmse_per_hour.query(f'model == "{m}" & experiment == "{e}"'), 
                             label, 'rmse', H=H, color=flux_colors[j], z=z)
            plot_performance(ax[1, i], pcc_per_hour.query(f'model == "{m}" & experiment == "{e}"'), 
                             label, 'pcc', H=H, color=flux_colors[j], z=z)

    ax[1,i].set_xlabel('forcasting horizon [h]', fontsize=14)
    
ax[0,0].set_ylabel('RMSE', fontsize=14)
ax[1,0].set_ylabel('Pearson r', fontsize=14)
ax[1,0].legend(loc='upper right', fontsize=12, 
            bbox_to_anchor=(0.98,1.13), 
               framealpha=1)
ax[0,0].set_title('Simulated data', fontsize=14)
ax[0,1].set_title('Radar data', fontsize=14)

for axis in ax.flatten():
    axis.tick_params(axis='both', which='major', labelsize=12)
    axis.grid(color = 'gray', linestyle = '-', alpha=0.2)

fig.subplots_adjust(wspace=0.15, hspace=0)
fig.align_ylabels(ax)
fig.savefig(osp.join(output_dir, f'fixed_rmse_pcc_per_hour{ext}.png'), bbox_inches='tight', dpi=200)